In [19]:
from keras.applications import inception_v3
from keras import backend as K

In [20]:
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [35]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]

fetch_loss_and_grads = K.function([dream], outputs)
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [44]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x*= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)

    return img

In [50]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10
base_image_path = 'invoker.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]

successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim/ (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

print(successive_shapes)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print ('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original_img = resize_img(original_img, shape)
    
    lost_detail = same_size_original_img - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
save_img(img, fname='final_dream.png')

[(480, 800), (342, 571), (244, 408)]
Processing image shape (244, 408)
...Loss value at 0 : 1.66842
...Loss value at 1 : 1.87864
...Loss value at 2 : 2.42955
...Loss value at 3 : 3.14945
...Loss value at 4 : 3.82142
...Loss value at 5 : 4.54492
...Loss value at 6 : 5.22331
...Loss value at 7 : 5.93976
...Loss value at 8 : 6.64884
...Loss value at 9 : 7.32313
...Loss value at 10 : 8.03306
...Loss value at 11 : 8.65654
...Loss value at 12 : 9.30137
...Loss value at 13 : 9.83113
Processing image shape (342, 571)
...Loss value at 0 : 3.19986
...Loss value at 1 : 4.61719
...Loss value at 2 : 5.75622
...Loss value at 3 : 6.75738
...Loss value at 4 : 7.66669
...Loss value at 5 : 8.50631
...Loss value at 6 : 9.29299
Processing image shape (480, 800)
...Loss value at 0 : 3.29671
...Loss value at 1 : 4.4772
...Loss value at 2 : 5.56061
...Loss value at 3 : 6.56671
...Loss value at 4 : 7.51786
...Loss value at 5 : 8.42448
...Loss value at 6 : 9.27976


In [49]:
img = preprocess_image('pa.jpg')
model.predict(img, 1)

array([[[[ 0.        ,  0.50698996,  1.12446356, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.48044124,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.05864118,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ..., 
         [ 0.31237513,  0.        ,  0.        , ...,  0.10285063,
           0.        ,  0.        ],
         [ 0.95206565,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.25396895, ...,  0.        ,
           0.        ,  0.        ]],

        [[ 0.        ,  0.14586219,  0.05596454, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ..., 
         [ 0.00698549,  0.172984